# Tarea adicional – Práctica de Aprendizaje Automático

**Asignatura:** Aprendizaje Automático  
**Curso:** 2024–2025  
**Trabajo práctico individual/grupal**  
**Alumnas:** [Elena Recio Álvarez - 100495725], [Alejandra Castuera García - 100499172]  
**Tarea de elección abierta:** Selección de atributos con `SelectKBest`  


## Justificación

Como tarea adicional, hemos decidido aplicar un proceso de selección de atributos utilizando `SelectKBest` con la función `mutual_info_classif`.

Este método permite seleccionar las variables más relevantes según la información mutua que tienen con la variable objetivo.  
Nuestro objetivo con esta técnica es:

- Reducir la cantidad de atributos utilizados
- Comprobar si se puede mantener un rendimiento similar con menos variables
- Simplificar el modelo y acelerar el entrenamiento

Es una técnica especialmente útil cuando se trabaja con muchos atributos o modelos más pesados como las SVM.



In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, balanced_accuracy_score

# Carga del dataset
df = pd.read_csv("attrition_availabledata_07.csv.gz")
X = df.drop(columns=["Attrition"])
y = df["Attrition"]

# División train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1/3, stratify=y, random_state=1725
)

# Columnas numéricas y categóricas
columnas_num = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
columnas_cat = X_train.select_dtypes(include="object").columns.tolist()

# Preprocesado
preprocessor = ColumnTransformer([
    ("num", Pipeline([
        ("imp", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]), columnas_num),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]), columnas_cat)
])

# Pipeline con selección de atributos y modelo
pipe_kbest = Pipeline([
    ("preprocessor", preprocessor),
    ("select", SelectKBest(score_func=mutual_info_classif, k=20)),
    ("modelo", SVC(kernel="rbf", C=10, gamma=0.01))
])

# Evaluación con validación cruzada
cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=1725)
scorer = make_scorer(balanced_accuracy_score)

score_kbest = cross_val_score(pipe_kbest, X_train, y_train, cv=cv_inner, scoring=scorer).mean()
print("SVM con SelectKBest (20 variables) → Balanced Accuracy:", round(score_kbest, 4))


SVM con SelectKBest (20 variables) → Balanced Accuracy: 0.5362


## Conclusión

La selección de variables con `SelectKBest` y `mutual_info_classif` ha reducido el número de atributos a solo 20, sin que eso afecte negativamente al rendimiento del modelo.

La SVM con kernel RBF ha mantenido un nivel de balanced accuracy similar al original, lo que indica que muchas variables del dataset no eran realmente relevantes.

Esta técnica ha demostrado ser útil para simplificar el modelo y podría aplicarse fácilmente si el dataset fuera más grande o si se quisiera acelerar el entrenamiento en futuras versiones del modelo.
